In [1]:
print("ok")

ok


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [5]:
PINCONE_API_KEY = "2761fe04-6a47-4657-ba1b-a9344ec1face"
PINECONE_API_ENV = "us-east-1"

In [6]:
#extract data from pdf 
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glab="*.pdf",
                    Loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents